# Cognitive Portfolio Optimization

## This notebook outlines steps to combine AI with Portfolio Optimization service

This notebook requires a tradable universe. The experimental version of the Portfolio Optimization service http://cloud.ibm.com/catalog/services/portfolio-optimization has a static data set for users to play with; a dynamic data set will be integrated in a future release. Note that much of the data in this sample set is _representative_ of actual data. 

The spreadsheet "Universe Data" contains all relevant information on the current sample data set.

## Step 1 - Load in Data Set

Load the file into the notebook. Apparently there's a command that simply inserts the code into your notebook that converts the csv file to a pandas dataframe!

In [1]:
from io import StringIO
import requests
import json
import pandas as pd
import numpy as np
import spacy
import random

In [2]:
universe_data = pd.read_csv('Universe Data.csv')
universe_data.head()

,Ticker,Name,CUSIP,Sector,Geography,Asset Class,Last Close Price,One Month Return,Risk Score,Controversy,...,Social,Sustainability,Has Tobacco,Has Alcohol,Has Gambling,Has Military,Has Fossil Fuels,Benchmark - Conservative,Benchmark - Moderate,Benchmark - Aggressive
0,CX_IE00BY7QL619_NYQ,JOHNSON CONTROLS INTERNATIONAL PLC,00BY7QL61,Industrials,Domestic,Equity,44.00,0.007803,5,Average,...,High,Low,0,0,0,0,0,0.0,0.0,0.0
1,CX_US02079KAA51_USD,ALPHABET INC,02079KAA5,Information Technology,Domestic,Corporate Bonds,105.39,-0.000429,3,Low,...,Low,Average,0,0,0,0,0,0.0,0.0,0.0
2,CX_US031162BG42_USD,AMGEN INC,031162BG4,Health Care,Domestic,Corporate Bonds,106.49,0.000237,4,Average,...,High,Low,0,0,0,0,0,0.0,0.0,0.0
3,CX_US031162BK53_USD,Amgen 5.15% 11/15/2041,031162BK5,Health Care,Domestic,Corporate Bonds,116.05,0.004569,3,High,...,High,Average,0,0,0,0,0,0.0,0.0,0.0
4,CX_US03232PAD06_USD,AmSurg Corp 5.625% 07/15/2022,03232PAD0,Health Care,Domestic,Corporate Bonds,104.00,0.000594,3,Average,...,Average,Average,0,0,0,0,0,0.0,0.0,0.0


## Step 2 - Get Investor Profile and Preferences as Natural Text

Watson Natural Language Understanding service http://cloud.ibm.com/catalog/services/natural-language-understanding analyzes provided text (in text, url, or html format) for the specified semantic features.

It can extract entities, concepts, keywords, categories, relations, sentiment, and emotions from provided text.

Watson Language Classifier service also lets you build custom classifiers to classify texts provided that you have training data.

In [3]:
# try:
#     from watson_developer_cloud import NaturalLanguageUnderstandingV1 as NLU
#     from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, ConceptsOptions, CategoriesOptions, EmotionOptions, RelationsOptions, SemanticRolesOptions
# except:
#     !pip install watson_developer_cloud

In [4]:
# Insert your username and password
# apikey='sy3UugZgezzNAb8k_Ks42Eh_3VmE07wwE96m8Sxk8qXi'
# nlu = NLU(
#     iam_apikey=apikey,
#     version='2018-03-16'
# )

Create text (investor profile and preferences) to analyze.

In [5]:
investor_text='My name is Pam. I am 39. I work in health care sector. I would like to invest $30000 in my RRSP this year. I would like highly diversified portfolio. I do not like having tobacco stocks in my portfolio. I care a lot about sustainable development.'

From natural language understanding of the text we may try to figure out the following:
- **Holdings**: Where are we starting from? A blank slate? Or does the user have existing holdings? (*investor is starting from a blank portfolio and would like to invest $30000, portfolio construction problem instead of portfolio rebalancing*)
- **Benchmark**: If we're matching the properties of something, what are we matching? (*Aggressive, Moderate, or Conservative benchmark, Aggressive benchmark in this case as investor is 34 and investing for retirement (RRSP) - 30+ years till retirement, plus young person*)
- **Objective**: What are we trying to do? Are we minimizing or maximizing some property? Or are we matching a target? (*minimizing tracking error with respect to the benchmark, we can incorporate multiple objective functions later*)
- **Preferences**: What are the users unique preferences and constraints that we need to take into account when performing this calculation? Is there anything that can't be included? Do we need to have a specific weight at the end of our calculation? (*as investor wants highly diversified portfolio weights of each asset should be less than 5-10%, no military of fossil fuel assets in an optimal portfolio, high weight of sustainability assets, high weight of IT assets as investor works in IT*)

#### Concepts
This function uses the natural language understanding object to analyze the provided text for the top 10 concepts in it:

In [6]:
# response = nlu.analyze(text=investor_text, features=Features(concepts=ConceptsOptions(limit=10)))
# print(json.dumps(response.result, indent=2))

#### Keywords
This function uses the natural language understanding object to analyze the provided text for keywords in it:

In [7]:
# response = nlu.analyze(text=investor_text, features=Features(keywords=KeywordsOptions(emotion=True, sentiment=True)))
# print(json.dumps(response.result, indent=2))

#### Categories
This function uses the natural language understanding object to analyze the provided text for the categories that it fits into:

In [8]:
# response = nlu.analyze(text=investor_text, features=Features(categories=CategoriesOptions()))
# print(json.dumps(response.result, indent=2))

#### Emotions
This function uses the natural language understanding object to analyze the provided text to find the emotion that it is conveying:

In [9]:
# response = nlu.analyze(text=investor_text, features=Features(emotion=EmotionOptions(targets=['information technology','military'])))
# print(json.dumps(response.result, indent=2))

#### Entities
This function uses the natural language understanding object to analyze the provided text to find relevant entities in the text:

In [10]:
# all_feat = nlu.analyze(text=investor_text, features=Features(entities=EntitiesOptions(sentiment=True)))
# all_feat.result

In [11]:
# for i in range(len(all_feat.result['entities'])):
#        print(all_feat.result['entities'][i]['text'],all_feat.result['entities'][i]['type'])

In [12]:
# for i in range(len(all_feat.result['entities'])):
#     print(all_feat.result['entities'][i]['text'],all_feat.result['entities'][i]['type'])
#     if all_feat.result['entities'][i]['type'] == 'Quantity':
#         Cash = float( str(all_feat.result['entities'][0]['text']).replace('$',''))
#         print(Cash)

#### Relations
Recognize when two entities are related, and identify the type of relation:

In [13]:
# def getRelations(text):
#     response=nlu.analyze(
#         text=text,
#         features=Features(relations=RelationsOptions())
#     )
#     print('Raw Results: ')
#     print(json.dumps(response.result, indent=2))

In [14]:
# getRelations(investor_text)

#### Semantic Roles
This function uses the natural language understanding object to analyze the provided text to find relevant semantic roles in the text:

In [15]:
# def getSemanticRoles(text):
#     response=nlu.analyze(
#         text=text,
#         features=Features(semantic_roles=SemanticRolesOptions(entities=True,keywords=True,limit=50))
#     )
#     print('Raw Results: ')
#     print(json.dumps(response.result, indent=2))

In [16]:
# getSemanticRoles(investor_text)

# Step 3 - Inputs: Holdings, Benchmarks, Objectives, and Preferences

As with any optimization problem, there are a few things we need to determine before we get started:
- **Holdings**: Where are we starting from? A blank slate? Or does the user have existing holdings?
- **Benchmark**: If we're matching the properties of something, what are we matching?
- **Objective**: What are we trying to do? Are we minimizing or maximizing some property? Or are we matching a target?
- **Preferences**: What are the users unique preferences and constraints that we need to take into account when performing this calculation? Is there anything that can't be included? Do we need to have a specific weight at the end of our calculation?

In [17]:
#Initiliaze the request
optimization = {
    "portfolios": [],
    "objectives": [],
    "constraints": []
}

## Holdings

First we need to decide if we're starting from scratch, or if the user already has a portfolio of assets. We only need to specify the ID of the asset (our data set that we loaded above uses ticker) and the quantity in # of units. 

We also need to define how much cash - in addition to the assets in the portfolio - the client wants to add to this analysis.

We define this as an array of json objects:

In [18]:
##### PUT IN YOUR HOLDINGS HERE #####
Holdings = []
Holdings.append({"asset":"CX_US0533321024_NYQ","quantity":14}) #Example of a holding
Holdings.append({"asset":"CX_US0584981064_NYQ","quantity":162})
Holdings.append({"asset":"CX_US1696561059_NYQ","quantity":67})
Holdings.append({"asset":"CX_US1912161007_NYQ","quantity":68})
Holdings.append({"asset":"CX_US29379VAY92_USD","quantity":67})
Holdings.append({"asset":"CX_US30231GAN25_USD","quantity":68})
Holdings.append({"asset":"CX_US46120E6023_NSQ","quantity":13})
Holdings.append({"asset":"CX_US4878361082_NYQ","quantity":67})
Holdings.append({"asset":"CX_US5486611073_NYQ","quantity":67})
Holdings.append({"asset":"CX_US56585AAG76_USD","quantity":67})
Holdings.append({"asset":"CX_US651639AN69_USD","quantity":67})
Holdings.append({"asset":"CX_US70450Y1038_NSQ","quantity":67})
Holdings.append({"asset":"CX_US9100471096_NYQ","quantity":67})

#Example cash infusion
#Cash = 10000
Cash = 0
#####################################

#Initilialize the 'tradeable universe' - note that position units should be populated where assets are held.
tradeable_universe = {
    "name": "Universe",
    "type":"root",
    "holdings":[]
}
for index, row in universe_data.iterrows():
    holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
    if holding:
        tradeable_universe["holdings"].append(holding[0])
    else:
        tradeable_universe["holdings"].append({"asset":row["Ticker"],"quantity":0})

optimization["portfolios"].append(tradeable_universe)
# Debug
#print(json.dumps(optimization, indent=4, sort_keys=True))
#print("Holdings: " + str(Holdings))
#print("Cash: " + str(Cash))

## Benchmark

Next we need to pick a benchmark portfolio for our analysis. If we're simply optimizing to minimize variance/maximize return objectively, then this step can be skipped.

Instead, in our analysis we assume that our advisor is recommending an "off-the-shelf" portfolio that meets our risk and return objectives that we want to use, with some caveats. Here we select that benchmark by assigning the name to the Benchmark variable below.

Choices are: 
- Conservative
- Moderate
- Aggressive

In [19]:
#ENTER DETAILS HERE
Benchmark_name = "Aggressive"

#Assemble the benchmark portfolio using the above details:
Benchmark_col_name = "Benchmark - " + str(Benchmark_name)
benchmark = {
    "name": Benchmark_name,
    "type": "benchmark",
    "holdings": []
}
Benchmark_holdings = universe_data.filter(items=["Ticker","Last Close Price",Benchmark_col_name])[universe_data[Benchmark_col_name] != 0]
for index,row in Benchmark_holdings.iterrows():
    #Determine weight of each asset
    shares = row[Benchmark_col_name]
    holding = {"asset":row["Ticker"],"quantity":shares}
    benchmark["holdings"].append(holding)
    
optimization["portfolios"].append(benchmark)
#Debug
#print(Benchmark_holdings)
print(benchmark)

{'name': 'Aggressive', 'type': 'benchmark', 'holdings': [{'asset': 'CX_US4642872000_NYQ', 'quantity': 121.2807245}, {'asset': 'CX_US4642876142_NYQ', 'quantity': 164.3385374}, {'asset': 'CX_US4642878049_NYQ', 'quantity': 282.92544910000004}, {'asset': 'CX_US4642885135_NYQ', 'quantity': 225.73363430000003}, {'asset': 'CX_US46434G1031_NYQ', 'quantity': 191.46084630000001}]}


## Objective

Now we need to describe the overall mission of this analysis. We're going to hard-code this to be a minimization of variance of the returns of the portfolio - the standard [Markowitz approach](https://www.math.ust.hk/~maykwok/courses/ma362/07F/markowitz_JF.pdf).

For this type of analysis, the only parameters we need to be concerned with are:
- Whether or not we're trying to match the variance of returns relative to the benchmark, or simply minimizing it.
- The time step, which is contingent on the data set (in our case it's 30 days)

In [20]:
optimization["objectives"] = [{
       "sense": "minimize",
       "measure": "variance",
       "attribute": "return",
       "portfolio": "Universe",
       "TargetPortfolio": Benchmark_name,
       "timestep": 30,
       "description": "minimize tracking error squared (variance of the difference between Universe portfolio and Benchmark returns) at time 30 days"
}]

#Debug 
print(optimization["objectives"])

[{'sense': 'minimize', 'measure': 'variance', 'attribute': 'return', 'portfolio': 'Universe', 'TargetPortfolio': 'Aggressive', 'timestep': 30, 'description': 'minimize tracking error squared (variance of the difference between Universe portfolio and Benchmark returns) at time 30 days'}]


## Preferences

Now we get to the real meat of the analysis. This is where we can specify our custom constraints that will make this a _personalized_ analysis, resulting a portfolio with [as close to] identical properties (returns) as the benchmark portfolio, but with assets that meet the criteria specified below.

There's a lot of flexibility in how one would specify these constraints, but for the sake of this walkthrough, we're identifying three main types:

1. __Filtering__: Ensuring that a certain type of investment is not present in the portfolio (e.g. "sin stocks", military investments)
2. __ESG and Socially Responsible Investing Requirements__: Determine a minimum threshold for types of companies one would want included in a portfolio (e.g. I want my portfolio's average Governance score to be "High")
3. __Investment Allocation__: Determine specific weight requirements of the portfolio (e.g. 70% bonds, 30% stocks)

The user can also specify standard result requirements including:
- Allow/Prevent short-selling (e.g. weights can or can't go negative)
- Maximum allocation to any one security

In [21]:
# Predicting constraints of the investor_text using Text classifier
output_dir = "."
nlp = spacy.load(output_dir)
doc = nlp(investor_text)
textcat = nlp.get_pipe("textcat")
print(doc.cats)
textcat.predict([doc])

{'military': 0.0004289161879569292, 'alcohol': 0.0035928667057305574, 'tobacco': 0.8998746275901794, 'gambling': 4.539787187241018e-05, 'fossil fuels': 9.752401092555374e-05, 'information technology': 4.550219091470353e-05, 'health Care': 0.5000000596046448, 'government': 5.1934282964793965e-05, 'consumer staples': 4.539787187241018e-05, 'real estate': 4.953059760737233e-05}


(array([[4.2891619e-04, 3.5928667e-03, 8.9987463e-01, 4.5397872e-05,
         9.7524011e-05, 4.5502191e-05, 5.0000006e-01, 5.1934283e-05,
         4.5397872e-05, 4.9530598e-05]], dtype=float32),
 [array([], dtype=float32)])

In [22]:
sectors = ["information technology","health Care","government","consumer staples","real estate"]

types_of_constraints = ["Has Tobacco","Has Alcohol","Has Gambling","Has Military","Has Fossil Fuels"]
Filtering_Constraints = []
for h in doc.cats:
    if doc.cats[h] >= 0.5 and h not in sectors:
        Filtering_Constraints.append("Has "+h.capitalize())

print(Filtering_Constraints)

['Has Tobacco']


In [23]:
##### CONSTRAINTS #####
#Uncomment the constraints you wish to apply:

#Filtering Constraints - "The final portfolio..."
# Filtering_Constraints = []
#Filtering_Constraints.append("Has Tobacco")
#Filtering_Constraints.append("Has Alcohol")
#Filtering_Constraints.append("Has Gambling")
#Filtering_Constraints.append("Has Military") #Ex) No military investments desired in the portfolio
# Filtering_Constraints.append("Has Fossil Fuels")
    
#ESG Constraints - Define both a field and an mean score desired (e.g. "Low","Average","High")
ESG_Constraints = []
#ESG_Constraints.append({"field":"Controversy","mean_score":"High"})
ESG_Constraints.append({"field":"Environmental","mean_score":"High"})
#ESG_Constraints.append({"field":"Governance","mean_score":"High"})
#ESG_Constraints.append({"field":"Social","mean_score":"High"})
#ESG_Constraints.append({"field":"Sustainability","mean_score":"High"})
                       
#Investment Allocation Constraints - I want to allocate [allocation]% of my portfoilio to be [value] identified by the [field] of the assets.
#inequality can be "greater-or-equal","less-or-equal" or "equal"
Allocation_Constraints = []
#Allocation_Constraints.append({"field":"ActiveOrPassive","value":"Passive","allocation":.1,"inequality":"greater-or-equal"}) #Field values are "Active" or "Passive"
#Allocation_Constraints.append({"field":"Geography","value":"International","allocation":.1,"inequality":"less-or-equal"}) #Field values are "Domestic" or "International"
#Allocation_Constraints.append({"field":"Asset Class","value":"Equity","allocation":.1,"inequality":"equal"}) #Field values are "Equity","Corporate Bonds","Municipal Bonds","Mortages","Mixed","Money Market", or "Commodities"
#Allocation_Constraints.append({"field":"Sector","value":"Information Technology","allocation":.1,"inequality":"greater-or-equal"}) #Field values are..
#"Consumer Discretionary","Consumer Staples","Energy","Financials","Health Care","Industrials","Information Technology","Materials","Real Estate","Utilities","Commodity","Government", or "Diversified"
Allocation_Constraints.append({"field":"Sector","value":"Industrials","allocation":.3,"inequality":"less-or-equal"}) #Field values are..

#Result requirements
AllowShortSales = False           #No short-selling
MaximumInvestmentWeight = .2      #20%
#MaximumNumberofPositions = 5     #Cardinality constraint on the portfolio

#Debug
print("Filtering Constraints: " + str(Filtering_Constraints))
print("ESG Constraints: " + str(ESG_Constraints))
print("Allocation Constraints: " + str(Allocation_Constraints))

Filtering Constraints: ['Has Tobacco']
ESG Constraints: [{'field': 'Environmental', 'mean_score': 'High'}]
Allocation Constraints: [{'field': 'Sector', 'value': 'Industrials', 'allocation': 0.3, 'inequality': 'less-or-equal'}]


In [24]:
#FILTERING CONTRAINTS
#Add subportfolio (how the optimization algorithm knows which asset has which property)
for f in Filtering_Constraints:
    #initialize the subportfolio
    subportfolio = {
        "ParentPortfolio":"Universe",
        "name":f,
        "type":"subportfolio",
        "holdings":[]
    }
    
    #Find all the assets that meet the criteria and add them to the subportfolio. Populate holdings quantity if available.
    assets = universe_data.filter(items=["Ticker",f])[universe_data[f] != 0]
    for index,row in assets.iterrows():  
        holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
        if holding:
            subportfolio["holdings"].append(holding[0])
        else:
            subportfolio["holdings"].append({"asset":row["Ticker"],"quantity":0})

    #Add subportfolio to list
    optimization["portfolios"].append(subportfolio)          
            
    #Add constraint to list
    optimization["constraints"].append({
        "attribute":"weight",
        "portfolio":f,
        "InPortfolio":"Universe",
        "relation":"equal",
        "constant":0.0,
        "description":"Excluding all securities which have the property " + f + "."
    })
    
#Debug
print(json.dumps(subportfolio, indent=4, sort_keys=True))

{
    "ParentPortfolio": "Universe",
    "holdings": [
        {
            "asset": "CX_US4642872000_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642872422_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642874402_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642874659_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642875078_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642875987_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642876142_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642878049_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642882819_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642886877_NYQ",
            "quantity": 0
        }
    ],
    "name": "Has Tobacco",
  

In [25]:
#ESG CONSTRAINTS
#Add subportfolio (how the optimization algorithm knows which asset has which property)
for e in ESG_Constraints:
    #initialize the subportfolio
    subportfolio = {
        "ParentPortfolio":"Universe",
        "name": e["mean_score"] + e["field"],
        "type":"subportfolio",
        "holdings":[]
    }
    
    #Find all the assets that meet the criteria and add them to the subportfolio. Populate holdings quantity if available.
    assets = universe_data.filter(items=["Ticker",e["field"]])[universe_data[e["field"]] == e["mean_score"]]
    for index,row in assets.iterrows():  
        holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
        if holding:
            subportfolio["holdings"].append(holding[0])
        else:
            subportfolio["holdings"].append({"asset":row["Ticker"],"quantity":0})

    #Add subportfolio to list
    optimization["portfolios"].append(subportfolio)          
            
    #Add constraint to list
    optimization["constraints"].append({
        "attribute":"weight",
        "portfolio":e["mean_score"] + e["field"],
        "InPortfolio":"Universe",
        "relation":"greater-or-equal",
        "constant":.5,
        "description":"Creating an average " + e["field"] + " score of " + e["mean_score"] + "."
    })
    
#Debug
print(json.dumps(subportfolio, indent=4, sort_keys=True))

{
    "ParentPortfolio": "Universe",
    "holdings": [
        {
            "asset": "CX_US03232PAD06_USD",
            "quantity": 0
        },
        {
            "asset": "CX_US0533321024_NYQ",
            "quantity": 14
        },
        {
            "asset": "CX_US1696561059_NYQ",
            "quantity": 67
        },
        {
            "asset": "CX_US4642871762_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642872000_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642874402_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642874576_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642884146_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642885135_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US4642886463_NYQ",
            "quantity": 0
        },
        {
            "asset": "C

In [26]:
#ALLOCATION CONSTRAINTS
#Add subportfolio (how the optimization algorithm knows which asset has which property)
for a in Allocation_Constraints:
    #initialize the subportfolio
    subportfolio = {
        "ParentPortfolio":"Universe",
        "name": a["value"],
        "type":"subportfolio",
        "holdings":[]
    }
    
    #Find all the assets that meet the criteria and add them to the subportfolio. Populate holdings quantity if available.
    assets = universe_data.filter(items=["Ticker",a["field"]])[universe_data[a["field"]] == a["value"]]
    for index,row in assets.iterrows():  
        holding = [h for h in Holdings if h["asset"] == row["Ticker"]]
        if holding:
            subportfolio["holdings"].append(holding[0])
        else:
            subportfolio["holdings"].append({"asset":row["Ticker"],"quantity":0})

    #Add subportfolio to list
    optimization["portfolios"].append(subportfolio)          
            
    #Add constraint to list
    optimization["constraints"].append({
        "attribute":"weight",
        "portfolio":a["value"],
        "InPortfolio":"Universe",
        "relation":a["inequality"],
        "constant":a["allocation"],
        "description":"Weight of " + a["value"] + " in the portfolio should be " + a["inequality"] + " to " + str(a["allocation"]) + "."
    })
    
#Debug
print(json.dumps(subportfolio, indent=4, sort_keys=True))

{
    "ParentPortfolio": "Universe",
    "holdings": [
        {
            "asset": "CX_IE00BY7QL619_NYQ",
            "quantity": 0
        },
        {
            "asset": "CX_US651639AN69_USD",
            "quantity": 67
        },
        {
            "asset": "CX_US9100471096_NYQ",
            "quantity": 67
        }
    ],
    "name": "Industrials",
    "type": "subportfolio"
}


In [27]:
#RESULT REQUIREMENTS

#No short-sale restriction
try:
    if AllowShortSales == False:
        optimization["constraints"].append({
           "attribute":"weight",
           "relation":"greater-or-equal",
           "members":"Universe",
           "constant":0,
           "description":"no short-sales for assets in Universe portfolio" 
        })
except:
    print("Short sales allowed")

#Maximum individual investment weight
try:
    optimization["constraints"].append({
       "attribute":"weight",
       "relation":"less-or-equal",
       "members":"Universe",
       "constant":MaximumInvestmentWeight,
       "description":"Weight of each asset from the Universe portfolio does not exceed " + str(MaximumInvestmentWeight*100) + "%."
    })
except:
    print("No maximum investment weight.")

#Maximum number of trades/positions
try:
    optimization["constraints"].append({
        "attribute": "count", 
        "relation": "less-or-equal", 
        "constant": MaximumNumberofPositions })
except:
    print("No maximum number of positions.")

#Minimum number of trades/positions
try:
    optimization["constraints"].append({
        "attribute": "count", 
        "relation": "greater-or-equal", 
        "constant": MinimumNumberofPositions })
except:
    print("No minimum number of positions.")

#Cash infusions
try:
    optimization["constraints"].append({
        "attribute:": "value",
        "portfolio": "Universe",
        "cashadjust": Cash,
        "description": "cash inflow of " + str(Cash) +" monetary units to the Universe portfolio"})
except:
    print("No cash infusions.")
    
# Debug
print(json.dumps(optimization["constraints"], indent=4, sort_keys=True))

No maximum number of positions.
No minimum number of positions.
[
    {
        "InPortfolio": "Universe",
        "attribute": "weight",
        "constant": 0.0,
        "description": "Excluding all securities which have the property Has Tobacco.",
        "portfolio": "Has Tobacco",
        "relation": "equal"
    },
    {
        "InPortfolio": "Universe",
        "attribute": "weight",
        "constant": 0.5,
        "description": "Creating an average Environmental score of High.",
        "portfolio": "HighEnvironmental",
        "relation": "greater-or-equal"
    },
    {
        "InPortfolio": "Universe",
        "attribute": "weight",
        "constant": 0.3,
        "description": "Weight of Industrials in the portfolio should be less-or-equal to 0.3.",
        "portfolio": "Industrials",
        "relation": "less-or-equal"
    },
    {
        "attribute": "weight",
        "constant": 0,
        "description": "no short-sales for assets in Universe portfolio",
        "me

## The Assembled Request:

We've assembled our payload! Below is what it looks like:

In [28]:
print(json.dumps(optimization, indent=4, sort_keys=True))

{
    "constraints": [
        {
            "InPortfolio": "Universe",
            "attribute": "weight",
            "constant": 0.0,
            "description": "Excluding all securities which have the property Has Tobacco.",
            "portfolio": "Has Tobacco",
            "relation": "equal"
        },
        {
            "InPortfolio": "Universe",
            "attribute": "weight",
            "constant": 0.5,
            "description": "Creating an average Environmental score of High.",
            "portfolio": "HighEnvironmental",
            "relation": "greater-or-equal"
        },
        {
            "InPortfolio": "Universe",
            "attribute": "weight",
            "constant": 0.3,
            "description": "Weight of Industrials in the portfolio should be less-or-equal to 0.3.",
            "portfolio": "Industrials",
            "relation": "less-or-equal"
        },
        {
            "attribute": "weight",
            "constant": 0,
            "descrip

# Step 4 - Submit the Request

Now that we have the payload assembled, we issue a [requests](http://docs.python-requests.org/en/master/) command to the Portfolio Optimization service running on the cloud. You'll want to provision your own copy of the application [which can be done here on Bluemix](https://console.bluemix.net/catalog/services/portfolio-optimization?env_id=ibm:yp:us-south). Fill in your uri and API credentials below before issuing the command.

In [29]:
##### ENTER YOUR PORTFOLIO OPTIMZIATION SERVICE CREDENTIALS #####
uri = "fss-analytics.mybluemix.net"
api_key = "7e8dcfd7e2fbe6661636a8b3a479a5491546172f8d8c0fd6eabc5e837b66c0fb8b54a6604835e9e49685b29d5a509918b068809af2879963294e8c6b7eefcbd345147370d4ee7829b8abf68f07712b27efe998baae301a2ac45e613097bd1e0e8a94e5847a047ead8468f92802ee90cbb3603d1327591a27922ac1dcea75ef4f"
#################################################################

url = "https://" + uri + "/api/v1/optimization/portfolio/construct"
headers = {'content-type': 'application/json', 'accept': 'application/json', 'X-IBM-Access-Token': api_key}
data = json.dumps(optimization)

import requests
r = requests.post(url,data=data,headers=headers)

print(r.text)


{
 "Holdings": [
  {
   "Asset": "CX_IE00BY7QL619_NYQ",
   "Quantity": 0,
   "OptimizedTrade": 171.707496399948,
   "OptimizedQuantity": 171.707496399948
  },
  {
   "Asset": "CX_US02079KAA51_USD",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "CX_US031162BG42_USD",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "CX_US031162BK53_USD",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "CX_US03232PAD06_USD",
   "Quantity": 0,
   "OptimizedTrade": 0,
   "OptimizedQuantity": 0
  },
  {
   "Asset": "CX_US0533321024_NYQ",
   "Quantity": 14,
   "OptimizedTrade": 20.4443820425064,
   "OptimizedQuantity": 34.444382042506405
  },
  {
   "Asset": "CX_US0584981064_NYQ",
   "Quantity": 162,
   "OptimizedTrade": -153.277042019289,
   "OptimizedQuantity": 8.722957980710987
  },
  {
   "Asset": "CX_US1696561059_NYQ",
   "Quantity": 67,
   "OptimizedTrade": -67,
   "OptimizedQuant

# Step 5 - Viewing your Results

So let's take a look at the results of our optimization calculation:

In [30]:
#Initialize Portfolio and field names
new_portfolio = []
fields = ["Name","Last Close Price","One Month Return"]

#Gather fields
for f in Filtering_Constraints:
    fields.append(f)
for e in ESG_Constraints:
    fields.append(e["field"])
for a in Allocation_Constraints:
    fields.append(a["field"])
    
#Assemble the data frame
for i in json.loads(r.text)["Holdings"]:
    if i["OptimizedQuantity"] != 0:
        security_data = universe_data.filter(items=fields)[universe_data["Ticker"] == i["Asset"]]
        security_data = security_data.values.tolist()[0]
        security_data.append(i["OptimizedQuantity"] * security_data[1])
        total_data = (str(i["Asset"]),float(i["OptimizedQuantity"])) + tuple(security_data)
        new_portfolio.append(total_data)
        
fields = ["Ticker","Quantity"]+fields+["Total Value"]
    
#Debug
#print(fields)
#print(new_portfolio)

In [31]:
OptimizedPortfolio = pd.DataFrame(new_portfolio, columns=fields)
OptimizedPortfolio

,Ticker,Quantity,Name,Last Close Price,One Month Return,Has Tobacco,Environmental,Sector,Total Value
0,CX_IE00BY7QL619_NYQ,171.707496,JOHNSON CONTROLS INTERNATIONAL PLC,44.00,0.007803,0,Average,Industrials,7555.129842
1,CX_US0533321024_NYQ,34.444382,AUTOZONE INC,495.69,-0.012556,0,High,Consumer Discretionary,17073.735735
2,CX_US0584981064_NYQ,8.722958,BALL CORP,42.42,-0.011617,0,Average,Materials,370.027878
3,CX_US4642885135_NYQ,225.754898,iShares iBoxx $ High Yield Corporate Bond ETF,88.60,0.000763,0,High,Diversified,20001.884000
4,CX_US46434G1031_NYQ,247.541456,iShares Core MSCI Emerging Markets,52.23,0.008036,0,High,Diversified,12929.090265
5,CX_US5486611073_NYQ,100.033743,LOWES COMPANIES INC,75.96,0.005499,0,Average,Consumer Discretionary,7598.563132
6,CX_US56585AAG76_USD,140.197937,MARATHON PETROLEUM CORP,101.27,0.001844,0,Low,Energy,14197.845051
7,CX_US70450Y1038_NSQ,190.919966,PAYPAL HOLDINGS INC,58.96,0.019552,0,Low,Information Technology,11256.641217
8,CX_US9100471096_NYQ,114.404346,UNITED CONTINENTAL HOLDINGS INC,78.90,0.017505,0,Low,Industrials,9026.502881


## How close did we match the risk of the benchmark?

In [32]:
print("Tracking error between returns of the benchmark and the optimized portfolio is: %.2f%%" % 
      (100*(json.loads(r.text)["Metadata"]["ObjectiveValue"])**0.5))

Tracking error between returns of the benchmark and the optimized portfolio is: 2.04%


## Checking that the constraints are met:

In [33]:
#Sum up quantity * price
portfolio_value = 0
for j in new_portfolio:
    portfolio_value += j[-1]
print("Total Portfolio Value: " + str(portfolio_value))

#We use a pandas dataframe to do our aggregation analysis:
for f in fields[5:-1]:
    print("Aggregation: " + f)
    elements = set(OptimizedPortfolio[f])
    for e in elements:
        aggr = OptimizedPortfolio.filter(items=fields)[OptimizedPortfolio[f] == e]
        aggr_pct = aggr["Total Value"].sum() / portfolio_value
        print("  Total allocation to " + str(e) + " is " + str(aggr_pct*100) + "%.")

Total Portfolio Value: 100009.42000000001
Aggregation: Has Tobacco
  Total allocation to 0 is 99.99999999999999%.
Aggregation: Environmental
  Total allocation to High is 50.000000000000036%.
  Total allocation to Low is 34.4777413454537%.
  Total allocation to Average is 15.522258654546253%.
Aggregation: Sector
  Total allocation to Diversified is 32.92787245976431%.
  Total allocation to Information Technology is 11.255580941277838%.
  Total allocation to Consumer Discretionary is 24.66997495518126%.
  Total allocation to Industrials is 16.580070879896102%.
  Total allocation to Materials is 0.3699930241988805%.
  Total allocation to Energy is 14.196507739681582%.


In [34]:
OptimizedPortfolio['Weight'] = OptimizedPortfolio.apply(lambda row: row['Quantity'] * row['Last Close Price'] / portfolio_value, axis=1)
OptimizedPortfolio

,Ticker,Quantity,Name,Last Close Price,One Month Return,Has Tobacco,Environmental,Sector,Total Value,Weight
0,CX_IE00BY7QL619_NYQ,171.707496,JOHNSON CONTROLS INTERNATIONAL PLC,44.00,0.007803,0,Average,Industrials,7555.129842,0.075544
1,CX_US0533321024_NYQ,34.444382,AUTOZONE INC,495.69,-0.012556,0,High,Consumer Discretionary,17073.735735,0.170721
2,CX_US0584981064_NYQ,8.722958,BALL CORP,42.42,-0.011617,0,Average,Materials,370.027878,0.003700
3,CX_US4642885135_NYQ,225.754898,iShares iBoxx $ High Yield Corporate Bond ETF,88.60,0.000763,0,High,Diversified,20001.884000,0.200000
4,CX_US46434G1031_NYQ,247.541456,iShares Core MSCI Emerging Markets,52.23,0.008036,0,High,Diversified,12929.090265,0.129279
5,CX_US5486611073_NYQ,100.033743,LOWES COMPANIES INC,75.96,0.005499,0,Average,Consumer Discretionary,7598.563132,0.075978
6,CX_US56585AAG76_USD,140.197937,MARATHON PETROLEUM CORP,101.27,0.001844,0,Low,Energy,14197.845051,0.141965
7,CX_US70450Y1038_NSQ,190.919966,PAYPAL HOLDINGS INC,58.96,0.019552,0,Low,Information Technology,11256.641217,0.112556
8,CX_US9100471096_NYQ,114.404346,UNITED CONTINENTAL HOLDINGS INC,78.90,0.017505,0,Low,Industrials,9026.502881,0.090257


In [35]:
OptimizedPortfolio.groupby("Environmental")["Weight"].sum()

Environmental
Average    0.155223
High       0.500000
Low        0.344777
Name: Weight, dtype: float64

In [36]:
OptimizedPortfolio.groupby("Sector")["Weight"].sum()

Sector
Consumer Discretionary    0.246700
Diversified               0.329279
Energy                    0.141965
Industrials               0.165801
Information Technology    0.112556
Materials                 0.003700
Name: Weight, dtype: float64

In [38]:
OptimizedPortfolio.groupby("Has Tobacco")["Weight"].sum()

Has Tobacco
0    1.0
Name: Weight, dtype: float64

## Compute portfolio expected return:

In [39]:
OptimizedPortfolio['Return'] = OptimizedPortfolio.apply(lambda row: row['One Month Return'] * row['Weight'], axis=1)

In [40]:
print("Portfolio Return: %.5f%%" % (100*OptimizedPortfolio["Return"].sum()))

Portfolio Return: 0.40547%


In [41]:
print("Annualized Portfolio Return: %.5f%%" % (100*12*OptimizedPortfolio["Return"].sum()))

Annualized Portfolio Return: 4.86566%


## Compute portfolio variance and tracking error:

In [42]:
import numpy as np

Read covariance matrix

In [43]:
df_cov = pd.read_csv('Covar_Universe_Data.csv', sep=',', header=None)
cov_matr = df_cov.values

Get holdings of initial and optimal portfolios as well as the benchmark portfolio

In [44]:
x_opt = []
x_init = []
for i in json.loads(r.text)["Holdings"]:
    x_opt.append(float(i["OptimizedQuantity"]))
    x_init.append(float(i["Quantity"]))
x_opt = np.array(x_opt)
x_init = np.array(x_init)

In [45]:
x_bench = universe_data[Benchmark_col_name].values

Compute portfolio values

In [46]:
val = universe_data["Last Close Price"].values
ret = universe_data["One Month Return"].values
print(ret)

[ 7.8034650e-03 -4.2937400e-04  2.3731300e-04  4.5688380e-03
  5.9383600e-04 -1.2555638e-02 -1.1617474e-02 -2.3754004e-02
  3.6032200e-03  5.6157880e-03  1.5441190e-03  2.6059887e-02
  6.4736240e-03  4.4599900e-04  7.2534720e-03  1.8470850e-03
  8.7190900e-04  2.2943600e-04 -1.4962400e-04  1.7528580e-03
  7.5125380e-03  5.9511270e-03  8.0877380e-03 -1.1267470e-02
  2.4601260e-03  6.1834000e-04  7.6251200e-04 -9.2442000e-04
  1.3753500e-04  1.1946200e-04 -3.3145500e-04  8.0363500e-03
  8.7496500e-04  5.4989580e-03  1.8439920e-03  5.7555540e-03
  1.9552159e-02  1.7505497e-02 -3.6790980e-03  7.4915200e-06]


In [47]:
portf_init_val = np.dot(val.T, x_init) # Initial portfolio value
portf_init_val

100009.41999999998

In [48]:
portf_opt_val = np.dot(val.T, x_opt) # Optimal portfolio value
portf_opt_val

100009.41999999998

In [49]:
np.allclose(portf_opt_val, OptimizedPortfolio["Total Value"].sum()) # Sanity check

True

Compute portfolio expected return, standard deviation and tracking error

In [50]:
w_opt   = x_opt * val / portf_opt_val
w_init  = x_init * val / portf_opt_val
w_bench = x_bench * val / portf_opt_val
# sharp_ratio = []
# l = np.prod(w_opt.shape)
# for i in np.arange(l):
#     if w_opt[i]!=0:
#        sharp_ratio.append(ret[i]/cov_matr[i][i])

# print(sharp_ratio)
# np.sort(sharp_ratio)
# print


In [51]:
ret_opt  = np.dot(ret.T, w_opt)
ret_init = np.dot(ret.T, w_init)

In [52]:
print("Expected monthly return of optimal portfolio: % .5f%%" % (100*ret_opt))
print("Expected monthly return of initial portfolio: % .5f%%" % (100*ret_init))

Expected monthly return of optimal portfolio:  0.40547%
Expected monthly return of initial portfolio: -0.13031%


In [53]:
print("Expected annual return of optimal portfolio: % .5f%%" % (12*100*ret_opt))
print("Expected annual return of initial portfolio: % .5f%%" % (12*100*ret_init))

Expected annual return of optimal portfolio:  4.86566%
Expected annual return of initial portfolio: -1.56377%


In [54]:
std_opt  = np.dot(np.dot(w_opt.T, cov_matr), w_opt)**0.5
std_init = np.dot(np.dot(w_init.T, cov_matr), w_init)**0.5

In [55]:
print("Standard deviation of optimal portfolio: %.8f" % std_opt)
print("Standard deviation of initial portfolio: %.8f" % std_init)

Standard deviation of optimal portfolio: 0.02765115
Standard deviation of initial portfolio: 0.02941583


In [56]:
tr_err_opt  = np.dot(np.dot((w_opt-w_bench).T, cov_matr), (w_opt-w_bench))**0.5
tr_err_init = np.dot(np.dot((w_init-w_bench).T, cov_matr), (w_init-w_bench))**0.5

In [57]:
print("Tracking error of optimal portfolio: %.8f" % tr_err_opt)
print("Tracking error of initial portfolio: %.8f" % tr_err_init)

Tracking error of optimal portfolio: 0.02036029
Tracking error of initial portfolio: 0.03596899


In [58]:
print("Number of assets in optimal portfolio: %d" % np.count_nonzero(w_opt))
print("Number of assets in initial portfolio: %d" % np.count_nonzero(w_init))

Number of assets in optimal portfolio: 9
Number of assets in initial portfolio: 13


In [66]:
# Load time series data of stock prices
stock_prices = pd.read_csv("stock_prices.csv")
print(type(stock_prices.columns))
print(type(OptimizedPortfolio))
subset_stock_prices = {}

subset_stock_prices["date"] = (stock_prices["date"])

for i, j in OptimizedPortfolio.iterrows(): 
    for h in stock_prices.columns.tolist():
        if h==j["Name"]:
            subset_stock_prices[h] = stock_prices[h]
            

print((subset_stock_prices))



<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.frame.DataFrame'>
{'date': 0      2019-01-30
1      2019-01-31
2      2019-02-01
3      2019-02-04
4      2019-02-05
          ...    
247    2020-01-23
248    2020-01-24
249    2020-01-27
250    2020-01-28
251    2020-01-29
Name: date, Length: 252, dtype: object, 'JOHNSON CONTROLS INTERNATIONAL PLC': 0      32.243599
1      32.906200
2      32.399506
3      32.954926
4      33.169292
         ...    
247    41.750000
248    41.349998
249    40.299999
250    40.810001
251    40.490002
Name: JOHNSON CONTROLS INTERNATIONAL PLC, Length: 252, dtype: float64, 'AUTOZONE INC': 0       843.169983
1       847.340027
2       849.909973
3       869.440002
4       869.419983
          ...     
247    1166.000000
248    1140.229980
249    1114.489990
250    1107.579956
251    1096.510010
Name: AUTOZONE INC, Length: 252, dtype: float64, 'BALL CORP': 0      50.752544
1      51.843361
2      51.595444
3      51.545864
4      52.557350
      

In [61]:
#long-only portfolio that maximises the Sharpe ratio (a measure of risk-adjusted returns)
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# Read in price data
df = pd.DataFrame.from_dict(subset_stock_prices)

df.set_index('date',inplace=True)


# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)


# Optimise for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

{'JOHNSON CONTROLS INTERNATIONAL PLC': 0.10349, 'AUTOZONE INC': 0.09997, 'BALL CORP': 0.17808, 'iShares iBoxx $ High Yield Corporate Bond ETF': 0.61846, 'iShares Core MSCI Emerging Markets': 0.0, 'MARATHON PETROLEUM CORP': 0.0, 'PAYPAL HOLDINGS INC': 0.0}
Expected annual return: 17.5%
Annual volatility: 7.0%
Sharpe Ratio: 2.22


(0.17535559561669756, 0.07005548714904317, 2.217607812592585)

In [62]:
# Markowitz Critical Line Algorithm
from pypfopt.cla import CLA
cla = CLA(mu, S)
print(cla.max_sharpe())
cla.portfolio_performance(verbose=True)

{'JOHNSON CONTROLS INTERNATIONAL PLC': 0.08005728518612548, 'AUTOZONE INC': 0.07707886086786356, 'BALL CORP': 0.11306581236737707, 'iShares iBoxx $ High Yield Corporate Bond ETF': 0.722031251411065, 'iShares Core MSCI Emerging Markets': 0.0, 'MARATHON PETROLEUM CORP': 0.0, 'PAYPAL HOLDINGS INC': 0.0077667901675690775}
Expected annual return: 14.9%
Annual volatility: 5.9%
Sharpe Ratio: 2.18


(0.14897826308316797, 0.059197890153827926, 2.1787645260331603)

In [63]:
# Hierarchical Risk Parity optimisation method
from pypfopt.hierarchical_risk_parity import HRPOpt
# This function by default calculates the percentage change from the immediately previous row
returns = df.pct_change().dropna()
hrp = HRPOpt(returns)
weights = hrp.hrp_portfolio()
print(weights)
hrp.portfolio_performance(verbose=True)

{'AUTOZONE INC': 0.03597946036760566, 'BALL CORP': 0.03781008908255258, 'JOHNSON CONTROLS INTERNATIONAL PLC': 0.04970609321549348, 'MARATHON PETROLEUM CORP': 0.013683083555858415, 'PAYPAL HOLDINGS INC': 0.02414881675633326, 'iShares Core MSCI Emerging Markets': 0.06053147326975273, 'iShares iBoxx $ High Yield Corporate Bond ETF': 0.7781409837524039}
Expected annual return: 0.0%
Annual volatility: 0.3%
Sharpe Ratio: -5.62


(0.00044611223607726047, 0.003476619895993381, -5.624396209219637)

In [65]:
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
viewdict = {"AUTOZONE INC": 0.1, "BALL CORP": 0.030, "PAYPAL HOLDINGS INC": 0}
bl = BlackLittermanModel(S, absolute_views=viewdict)

rets = bl.bl_returns()
ef = EfficientFrontier(rets, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)
# OR use return-implied weights
# delta = black_litterman.market_implied_risk_aversion(df)
# bl.bl_weights(delta)
# weights = bl.clean_weights()

Expected annual return: 4.9%
Annual volatility: 20.8%
Sharpe Ratio: 0.14


/home/evilmorty/anaconda3/lib/python3.7/site-packages/pypfopt/black_litterman.py:173: UserWarning: Running Black-Litterman with no prior.
  warnings.warn("Running Black-Litterman with no prior.")


(0.04936228104635638, 0.2080918940094082, 0.14110247391486022)